In [1]:
#!/usr/bin/env python
# coding: utf-8
import json
import time
import requests
import pandas as pd    
from requests import get
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException,StaleElementReferenceException

In [2]:
def add_to_json(data, title, summary, url, category):
    """
    Function for adding data contents to a json file
    """
    data['Article'].append({
        'Title': title,
        'Content': summary,
        'Link': url,
        'Category': category
    })
    return(data['Article'])

def save_json(data, fname):
    """
    Function for saving json file
    """
    with open('{}.json'.format(fname), 'w') as outfile:
        json.dump(data, outfile)

Young Citizen Science portal - Citizen science https://en.wikipedia.org

# MSME micro, small, medium enterprises and entrepreneurship

In [3]:
links = {"https://ryadiybusiness.com/category/%d9%85%d8%b4%d8%a7%d8%b1%d9%8a%d8%b9/page/":7, 
         "https://ryadiybusiness.com/category/%d8%a7%d9%84%d8%aa%d8%b3%d9%88%d9%8a%d9%82-%d9%88%d8%a7%d9%84%d8%a5%d8%a8%d8%aa%d9%83%d8%a7%d8%b1/page":48,
         "https://ryadiybusiness.com/category/%d8%a7%d9%84%d8%ad%d8%a7%d8%b6%d9%86%d8%a7%d8%aa-%d9%88%d8%a7%d9%84%d9%85%d8%b3%d8%b1%d8%b9%d8%a7%d8%aa/page/":44,
         "https://ryadiybusiness.com/category/%d8%a7%d9%84%d8%aa%d9%85%d9%88%d9%8a%d9%84-%d9%88%d8%a7%d9%84%d8%a5%d8%b3%d8%aa%d8%ab%d9%85%d8%a7%d8%b1/page/":89}

categories = ["مشاريع", "التسويق والابتكار", "الحاضنات والمسرعات", "التمويل والاستثمار"]

data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)

l=0
for c in range(len(categories)):
    
    for i in range(1,list(links.values())[c]):
        
        driver.get(list(links.keys())[c]+"{}".format(i))
        soup=BeautifulSoup(driver.page_source,'html.parser')
        stories = soup.findAll("div", {"class": "td-block-span6"})

        for n in range(len(stories)):
            
            title = stories[n].find("h3", {"class":"entry-title td-module-title"}).find('a').text
            url = stories[n].find("h3", {"class":"entry-title td-module-title"}).find('a').attrs['href']
            driver.get(url)
            soup = BeautifulSoup(driver.page_source,'html.parser')
            div = soup.find("div", {"class": "td-post-content tagdiv-type"})
            ps = div.findAll('p')
            content = ""
            for p in ps:
                content += p.text
            category = categories[c]
            
            if content != "" and category != "":
                add_to_json(data, title, content, url, category)
                l += 1
    print(l)

save_json(data, '../Datasets/entrepreneurship/ryadibusiness')

46
501
922
1779


In [4]:
#df = pd.read_csv('../Datasets/ryadibusiness.csv')
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/entrepreneurship/ryadibusiness.csv", index=False)
df

,Title,Content,Link,Category
0,تقرير: تأثير فيروس كورونا على ريادة الأعمال في...,مع بدء تفشي فيروس كورونا خلال شهر مارس الماضي...,https://ryadiybusiness.com/%d8%aa%d9%82%d8%b1%...,مشاريع
1,4 مشاكل تهدد مستقبل ممنوح الفرنشايز,الامتياز التجاري (الفرنشايز) هو واحد من أفضل ...,https://ryadiybusiness.com/4-%d9%85%d8%b4%d8%a...,مشاريع
2,تدشين منظومة هاندي للمشاريع المنزلية,دشنت منظومة “هاندي” أول منظومة افتراضية للمش...,https://ryadiybusiness.com/handy-event/,مشاريع
3,لرواد الأعمال.. تعرف على أبرز الصناعات الواعدة...,حقق القطاع الصناعي تقدم كبير منذ إطلاق “رؤية ...,https://ryadiybusiness.com/%d9%84%d8%b1%d9%88%...,مشاريع
4,6 خطوات يجب القيام بها عند بدء مشروعك الجديد,هناك دائمًا قرارين مهمين لأي شخص يبدأ مشروعًا...,https://ryadiybusiness.com/6-%d8%ae%d8%b7%d9%8...,مشاريع
...,...,...,...,...
1774,عروض تمويل بـ 20 مليون ريال لاكتتاب “أرامكو” ا...,تفاوتت عروض تمويل الاكتتاب المقدمة من البنوك ا...,https://ryadiybusiness.com/%d8%b9%d8%b1%d9%88%...,التمويل والاستثمار
1775,«سامبا» ينضم إلى بوابة تمويل المنشآت الصغيرة و...,كشفت مجموعة سامبا المالية عن انضمامها إلى بواب...,https://ryadiybusiness.com/%d8%b3%d8%a7%d9%85%...,التمويل والاستثمار
1776,“إيكار” تبدأ أعمالها فى المملكة بعد اقفال المر...,أعلنت ‘إيكار‘ (ekar)، المشغل الأول والأكبر في ...,https://ryadiybusiness.com/%d8%a5%d9%8a%d9%83%...,التمويل والاستثمار
1777,تحديد النطاق السعري لسعر سهم “أرامكو السعودية...,كشفت شركة أرامكو اليوم الأحد عن النطاق السعري ...,https://ryadiybusiness.com/%d8%aa%d8%ad%d8%af%...,التمويل والاستثمار


In [5]:
df['Category'].value_counts()

التمويل والاستثمار    857
التسويق والابتكار     455
الحاضنات والمسرعات    421
مشاريع                 46
Name: Category, dtype: int64

In [6]:
data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get("https://waya.media/arabi/%d8%b4%d8%b1%d9%83%d8%a7%d8%aa-%d9%86%d8%a7%d8%b4%d8%a6%d8%a9/")

while True:
    try:
        showmore=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,'//*[@id="v/what-are-algorithms-panel"]/div[1]/div/div[6]/div/div[4]/button')))
        showmore.click()
    except TimeoutException:
        break
    except StaleElementReferenceException:
        break
        
for i in range(20):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)
    
soup=BeautifulSoup(driver.page_source,'html.parser')
media = soup.findAll("div", {"class": "post-media"})

l=0
for n in range(len(media)):
    
    title = media[n].find('img').attrs['alt']
    url = media[n].find('a').attrs['href']
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    div = soup.find("div", {"class": "entry-content"})
    ps = div.findAll('p')
    content = ""
    for p in ps:
        content += p.text
    if soup.find("span", {"class":"post-category"}) != None:
        category = soup.find("span", {"class":"post-category"}).find("span").text
        if content != "" and category != "":
            add_to_json(data, title, content, url, category)
            l += 1
print(l)
save_json(data, '../Datasets/entrepreneurship/waya')

140


In [7]:
#df = pd.read_csv('../Datasets/waya.csv')
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/entrepreneurship/waya.csv", index=False)
df

,Title,Content,Link,Category
0,إطلاق “أقول” أول تطبيق محادثات سعودي على الهوا...,أعلنت منصة “هلا یلا” السعودية، إطلاق تطبيق “أق...,https://waya.media/arabi/%d8%a5%d8%b7%d9%84%d8...,شركات ناشئة
1,الرئيس التنفيذي السابق في Just Eat يستثمر في e...,أعلنت المنيوز عن استثمار ديفيد باتريس الرئيس ا...,https://waya.media/arabi/%d8%a7%d9%84%d8%b1%d8...,شركات ناشئة
2,Starzplay تحصد تمويلا بالدين بقيمة ٢٥ مليون دو...,حصلت ستارزبلاي Starzplay منصة مشاهدة الفيديو ح...,https://waya.media/arabi/starzplay-%d8%aa%d8%a...,شركات ناشئة
3,“ماي سيارة” تجمع ٦٥٠ ألف دولار في جولة تمويل م...,أعلنت شركة “ماي سيارة” MySyara، ومقرها الإمارا...,https://waya.media/arabi/%d9%85%d8%a7%d9%8a-%d...,شركات ناشئة
4,“هلا یلا”.. أول منصة شاملة للترفيه والرياضة وا...,أراد مؤسسو المنصة أن يسهلوا تجربة الترفیه في ...,https://waya.media/arabi/%d9%87%d9%84%d8%a7-%d...,شركات ناشئة
...,...,...,...,...
135,"NowPay للمدفوعات الرقمية تحصد ٢,١ مليون دولار",حصلت شركة التكنولوجيا المالية المصرية”ناو باي”...,https://waya.media/arabi/nowpay-%d9%84%d9%84%d...,أخبار
136,رحلتي مع رواد الأعمال| التكلفة الحقيقة للوصول ...,إذا كنت تبني شركة ناشئة تتخطى قيمتها مليار دول...,https://waya.media/arabi/%d8%b1%d8%ad%d9%84%d8...,رأي
137,ما مستقبل الاستثمار في الشركات الناشئة اللبنانية؟,أثرت الأحداث الأخيرة من الاحتجاجات، وانتشار في...,https://waya.media/arabi/%d9%85%d8%a7-%d9%85%d...,تقارير
138,Flick تحصد مليون دولار في جولة ما قبل التمويل ...,حصل تطبيق فليك المصري لتحويل الأموال علي مليون...,https://waya.media/arabi/flick-%d8%aa%d8%ad%d8...,أخبار


In [8]:
df['Category'].value_counts()

شركات ناشئة    75
قطاعات         23
أخبار          14
مال             8
تقارير          6
رأي             5
تكنولوجيا       3
رواد            3
أسواق           1
Name: Category, dtype: int64

In [12]:
data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get("https://www.youm7.com/Tags/Index?id=149833&tag=%d8%b1%d9%8a%d8%a7%d8%af%d8%a9-%d8%a7%d9%84%d8%a7%d8%b9%d9%85%d8%a7%d9%84")
        
while True:
    try:
        element = driver.find_element(By.CLASS_NAME, 'showMoreNew')
    except:
        print("can't find the 'more' button anymore! Great")
        break;
    else:
        try:
            element.click()
            time.sleep(5)
        except:
            break;
    
soup=BeautifulSoup(driver.page_source,'html.parser')
media = soup.findAll("div", {"class": "col-xs-12 bigOneSec"})

l=0
for n in range(len(media)):
    
    title = media[n].find('h3').find('a').text
    url = "https://www.youm7.com" + media[n].find('h3').find('a').attrs['href']
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    #title = soup.find("div", {"class": "articlestyled__MastheadContent-rgo6b1-1 hHQxFA"}).find("h1").text
    div = soup.find("div", {"id": "articleBody"})
    ps = div.findAll('p')
    content = ""
    for p in ps:
        content += p.text
    category = "ريادة أعمال"
    if content != "" and category != "":
        add_to_json(data, title, content, url, category)
        l += 1
print(l)
save_json(data, '../Datasets/entrepreneurship/youm7')

96


In [13]:
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/entrepreneurship/youm7.csv", index=False)
df

,Title,Content,Link,Category
0,\n ...,\n\tكشف مشروع رواد 2030 التابع لوزارة التخطيط ...,https://www.youm7.com/story/2021/1/20/مشروع-رو...,ريادة أعمال
1,\n ...,"\n\t""شباب أسوان يبتكر"".. شعار رفعه الشباب والف...",https://www.youm7.com/story/2020/12/13/شباب-أس...,ريادة أعمال
2,\n ...,\n\tتناقش الدكتورة حنان يوسف فى الحلقة الجديدة...,https://www.youm7.com/story/2020/11/6/أنت-رائد...,ريادة أعمال
3,\n ...,\n\tيُعد القانون رقم 152 لسنة 2020 بشأن تنمية ...,https://www.youm7.com/story/2020/10/4/تعرف-على...,ريادة أعمال
4,\n ...,\n\tيواصل مشروع رواد 2030 التابع لوزارة التخطي...,https://www.youm7.com/story/2020/9/9/رواد-2030...,ريادة أعمال
...,...,...,...,...
218,\n القوى العاملة: تدريب الش...,\n أكد وزير القوي العاملة محمد سعفان، أن التنم...,https://www.youm7.com/story/2016/12/13/القوى-ا...,ريادة أعمال
219,\n وزير الاتصالات يبحث الإب...,\n\tعقد المهندس ياسر القاضى، وزير الاتصالات وت...,https://www.youm7.com/story/2016/12/12/وزير-ال...,ريادة أعمال
220,"\n الرئيس التنفيذى لـ""ايتيد...",\n\tكشفت المهندسة أسماء حسنى، الرئيس التنفيذى ...,https://www.youm7.com/story/2016/12/7/الرئيس-ا...,ريادة أعمال
221,\n رئيس جامعة بنها: المشروع...,\n\tأكد الدكتور السيد القاضى رئيس جامعة بنها، ...,https://www.youm7.com/story/2016/11/22/رئيس-جا...,ريادة أعمال


In [14]:
data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)

l=0    
for i in range(1,30):

    driver.get("https://jawlah.co/category/news/page/{}".format(i))
    soup = BeautifulSoup(driver.page_source,'html.parser')
    stories = soup.findAll("h2", {"class": "entry-title"})

    for n in range(len(stories)):

        title = stories[n].find('a').text
        url = stories[n].find('a').attrs['href']
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        div = soup.find("div", {"class": "entry-content"})
        ps = div.findAll('p')
        content = ""
        for p in ps:
            content += p.text
        category = "ريادة أعمال"

        if content != "" and category != "":
            add_to_json(data, title, content, url, category)
            l += 1
print(l)

save_json(data, '../Datasets/entrepreneurship/jawlah')

285


In [15]:
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/entrepreneurship/jawlah.csv", index=False)
df

,Title,Content,Link,Category
0,أيوا تعلن عن افتتاح أول متجر فعلي لها,أعلنت شركة أيوا للتجارة الإلكترونية للنظارات و...,https://jawlah.co/14368,ريادة أعمال
1,شركة AZOM تغلق جولة استثمارية بقيمة 36 مليون ر...,أعلنت شركة AZOM، المتخصصة في الإلكترونيات والب...,https://jawlah.co/14341,ريادة أعمال
2,منصة بث المحتوى Starzplay تحصل على تمويل بالدي...,حصلت منصة بث المحتوى المرئي في الشرق الأوسط وا...,https://jawlah.co/14361,ريادة أعمال
3,منصة MySyara تغلق جولة استثمارية بقيمة 650 ألف...,أعلنت منصة MySyara وهي منصة متخصصة في خدمات ال...,https://jawlah.co/14356,ريادة أعمال
4,شركة SparkPost المتخصصة في حملات البريد الإلكت...,جمعت SparkPost، وهي منصة متخصصة في حملات البري...,https://jawlah.co/14349,ريادة أعمال
...,...,...,...,...
280,منصة تروللي لتوصيل البقالة تحصل على استثمار بق...,حصلت منصة تروللي “Trolley” المصرية والمختصة بخ...,https://jawlah.co/12875,ريادة أعمال
281,صندوق الثروة السيادية الكويتي يقود استثمار بقي...,تمكنت Knotel، المتخصصة في توفير مساحات وحلول م...,https://jawlah.co/12873,ريادة أعمال
282,منصة نعناع تحصل على تمويل بقيمة 6.6 مليون دولا...,تمكنت منصة توصيل المقاضي “نعناع” من إنهاء جولة...,https://jawlah.co/12871,ريادة أعمال
283,منصة “شفاء” لطلب الدواء تحصل على تمويل تأسيسي ...,حصلت منصة “شفاء” التي تقدم خدمات طلب وتوفير ال...,https://jawlah.co/12869,ريادة أعمال


In [16]:
data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get("https://www.asharqbusiness.com/page/%D8%B1%D9%8A%D8%A7%D8%AF%D8%A9")
        
while True:
    try:
        element = driver.find_element(By.CLASS_NAME, 'btn-load-more')
    except:
        print("can't find the 'more' button anymore! Great")
        break;
    else:
        try:
            element.click()
            time.sleep(5)
        except:
            break;
    
soup=BeautifulSoup(driver.page_source,'html.parser')
stories = soup.findAll("h4", {"class": "heading--post"})

l=0
for n in range(len(stories)):
    
    title = stories[n].find('a').text
    url = "https://www.asharqbusiness.com" + stories[n].find('a').attrs['href']
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    div = soup.find("div", {"class": "col-12 col-md-9 col-xl-10 article--content"})
    if div != None:
        ps = div.findAll('p')
        content = ""
        for p in ps:
            content += p.text
        category = "ريادة أعمال"
        if content != "":
            add_to_json(data, title, content, url, category)
            l += 1
print(l)
save_json(data, '../Datasets/entrepreneurship/asharqbusiness')

can't find the 'more' button anymore! Great
34


In [17]:
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/entrepreneurship/asharqbusiness.csv", index=False)
df.head()

,Title,Content,Link,Category
0,\n هناء الرستما...,عيَّّن بنك أبوظبي الأول هناء الرستماني رئيساً ...,https://www.asharqbusiness.com/article/5372/هن...,ريادة أعمال
1,"\n ""كويشو"" الصي...","تسعى شركة "" كويشو تكنولوجي"" المنافس الرئيسي لش...",https://www.asharqbusiness.com/article/5039/كو...,ريادة أعمال
2,\n نصائح لرواد ...,"عندما كان ""ترينت جريفين براف"" يعمل كمديرٍ عامٍ...",https://www.asharqbusiness.com/article/3707/نص...,ريادة أعمال
3,"\n ""ستاركي"" لأج...","عندما بدأ براندون سواليتش العمل في شركة ""ستارك...",https://www.asharqbusiness.com/article/4307/ست...,ريادة أعمال
4,"\n ""الفتى الذهب...",عندما تعطَّلَت سيارة مويسيس كوسيو إيسبينوزا، و...,https://www.asharqbusiness.com/article/4124/ال...,ريادة أعمال


In [18]:
# Scroll + Button
data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get("https://www.raedaamal.com/category/news/")
  
l=0
while True:
    try:
        element = driver.find_element_by_id('load-more-button')
    except:
        l += 1
        print("can't find the 'more' button anymore! Great")
        if l > 2:
            break;
    else:
        try:
            element.click()
            time.sleep(5)
        except:
            break;
        
  
soup=BeautifulSoup(driver.page_source,'html.parser')
media = soup.findAll("h2", {"class": "entry-title"})

l=0
for n in range(len(media)):
    
    title = media[n].find('a').text
    url = media[n].find('a').attrs['href']
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    #title = soup.find("div", {"class": "articlestyled__MastheadContent-rgo6b1-1 hHQxFA"}).find("h1").text
    div = soup.find("div", {"class": "s-post-content s-post-small-el bb-mb-el"})
    ps = div.findAll('p')
    content = ""
    for p in ps:
        content += p.text
    category = "ريادة أعمال"
    if content != "":
        add_to_json(data, title, content, url, category)
        l += 1
print(l)
save_json(data, '../Datasets/entrepreneurship/raedaamal')

can't find the 'more' button anymore! Great
can't find the 'more' button anymore! Great
can't find the 'more' button anymore! Great
115


In [19]:
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/entrepreneurship/raedaamal.csv", index=False)
df

,Title,Content,Link,Category
0,منصة Botme المصرية المتخصصة في إنشاء روبوتات ا...,أعلنت منصة Botme المصرية المتخصصة في إنشاء روب...,https://www.raedaamal.com/2019/11/18/%d9%85%d9...,ريادة أعمال
1,شركة Trukker للشاحنات تحصل على تمويل بقيمة 23 ...,أعلنت شركة Trukker المختصة في مجال خدمات الشح...,https://www.raedaamal.com/2019/11/15/%d8%b4%d8...,ريادة أعمال
2,منصة Tabby للحلول الائتمانية تحصل على تمويل تأ...,أعلنت منصة Tabby الناشئة للحلول الائتمانية عن ...,https://www.raedaamal.com/2019/11/15/%d9%85%d9...,ريادة أعمال
3,منصة – يُداوي Yodawy – المصرية لخدمات طلب وتوص...,حصلت منصة يُداوي المصرية لتوصيل وإدارة طلبات ا...,https://www.raedaamal.com/2019/11/12/%d8%ad%d8...,ريادة أعمال
4,أهم 36 شركة ناشئة قامت بتغيير قواعد اللعبة في ...,أصدرت CB Insights تقريرها السنوي لعام 2019 ويح...,https://www.raedaamal.com/2019/11/03/%d8%a3%d9...,ريادة أعمال
...,...,...,...,...
110,السيارات الكهربية في شوارع مصر هذا الشهر,أعلنت شركة ريفولتا إيجيبت أنها قد اقتربت من ال...,https://www.raedaamal.com/2018/04/19/%d8%a7%d9...,ريادة أعمال
111,الذكاء الاصطناعي VS ذكاء الأبلة,شهد ثاني أيام جلسات “ملتقى صناعة الإبداع” والذ...,https://www.raedaamal.com/2018/04/18/%d8%a7%d9...,ريادة أعمال
112,سويفل تقدم طريقة أخرى لمعالجة معاناة المواصلات,في عام 2016، انضم مصطفى قنديل إلى كتيبة شركة ك...,https://www.raedaamal.com/2018/04/16/%d8%b3%d9...,ريادة أعمال
113,ماذا ستقدم NU TechSpace إلى رواد الأعمال؟,في مبادرة وتعاون بين جامعة النيل وأكاديمية ال...,https://www.raedaamal.com/2018/04/16/what-will...,ريادة أعمال


In [23]:
# check if the added posts are related

data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)

l=0    
for i in range(1,21):

    driver.get("https://www.egyentrepreneur.com/category/entrepreneurship/page/{}/".format(i))
    soup = BeautifulSoup(driver.page_source,'html.parser')
    stories = soup.findAll("h3", {"class": "jeg_post_title"})

    for n in range(len(stories)):

        title = stories[n].find('a').text
        url = stories[n].find('a').attrs['href']
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        div = soup.find("div", {"class": "entry-content no-share"})
        if div != None:
            ps = div.findAll('p')
            content = ""
            for p in ps:
                content += p.text
            category = "ريادة أعمال"

            if content != "" and category != "":
                add_to_json(data, title, content, url, category)
                l += 1
print(l)

save_json(data, '../Datasets/entrepreneurship/egyentrepreneur.json')

433


In [24]:
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/entrepreneurship/egyentrepreneur.csv", index=False)
df

,Title,Content,Link,Category
0,البنك الأهلي يستحوذ على 24% من أسهم رأسمال شرك...,وقع البنك الأهلي المصري عقدي شراء الأسهم واتفا...,https://www.egyentrepreneur.com/%d8%a7%d9%84%d...,ريادة أعمال
1,«كوفيد-19» يكتب تاريخًا جديدًا لقطاع ريادة الأ...,كتبت جائحة “كوفيد-19” تاريخا جديدا لقطاع ريادة...,https://www.egyentrepreneur.com/%d9%83%d9%88%d...,ريادة أعمال
2,مهاب مميش يفوز بجائزة الرواد لعام 2020,يسلم اليوم الدكتور خالد عبدالغفار وزير التعليم...,https://www.egyentrepreneur.com/%d9%85%d9%87%d...,ريادة أعمال
3,«رواد النيل» ترعى مشروع طلابي لتصميم وتصنيع سي...,أعلنت مبادرة رواد النيل الممولة من البنك المرك...,https://www.egyentrepreneur.com/%d8%b1%d9%88%d...,ريادة أعمال
4,الشركات الناشئة المصرية تجذب 3 مليارات جنيه خل...,قالت الدكتورة غادة خليل مدير مشروع رواد 2030، ...,https://www.egyentrepreneur.com/%d8%a7%d9%84%d...,ريادة أعمال
...,...,...,...,...
404,فتح باب التسجيل بـ«برنامج شيل انطلاقة» لريادة ...,فتح باب التسجيل ببرنامج شل انطلاقة مصر، وهي من...,https://www.egyentrepreneur.com/intlaagahshel/,ريادة أعمال
405,“أورنج” تفتح باب التسجيل بالمؤسم السادس لبرنام...,أطلقت شركة أورنج للاتصالات صفارة الأنطلاق للتس...,https://www.egyentrepreneur.com/blog-post_6-2/,ريادة أعمال
418,ما هي ريادة الأعمال Entrepreneurship,أصبحت ريادة الأعمال من أكثر المصطلحات المحيطة ...,https://www.egyentrepreneur.com/entrepreneurship/,ريادة أعمال
419,أكاديمية البحث العلمى: فتح باب التقدم في «مساب...,أعلنت أكاديمية البحث العملى، فتح باب التقدم لم...,https://www.egyentrepreneur.com/blog-post_28-2/,ريادة أعمال


In [25]:
data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)

l=0    
for i in range(1,13):

    driver.get("https://preneur-masr.com/tag/%d8%b1%d9%8a%d8%a7%d8%af%d8%a9-%d8%a7%d9%84%d8%a3%d8%b9%d9%85%d8%a7%d9%84/page/{}/".format(i))
    soup = BeautifulSoup(driver.page_source,'html.parser')
    stories = soup.findAll("div", {"class": "post-content"})

    for n in range(len(stories)):

        title = stories[n].find('h2').find('a').text
        url = "https://preneur-masr.com" + stories[n].find('h2').find('a').attrs['href']
        driver.get(url)
        
        soup = BeautifulSoup(driver.page_source,'html.parser')
        div = soup.find("div", {"class": "post-content"})
        ps = div.findAll('p')
        content = ""
        for p in ps:
            content += p.text
        category = "ريادة أعمال"

        if content != "":
            add_to_json(data, title, content, url, category)
            l += 1
print(l)

save_json(data, '../Datasets/entrepreneurship/preneur-masr')

110


In [26]:
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/entrepreneurship/preneur-masr.csv", index=False)
df

,Title,Content,Link,Category
0,الإمارات الخامسة عالمياً في ريادة الأعمال,صُنفَت دولة الإمارات في المركز الخامس على المس...,https://preneur-masr.com/news/%d8%a7%d9%84%d8%...,ريادة أعمال
1,باقي 4 أيام علي إنتهاء التقديم بمسابقة نواة 2020,مسابقة نواة أو جائزة الإبداع المصرية لريادة ال...,https://preneur-masr.com/news/%d8%a8%d8%a7%d9%...,ريادة أعمال
2,ICSB :: مستقبل ريادة الأعمال و MSME’s – مرحلة ...,يعقد الليلة الخميس 14 مايو 2020 في الساعه العا...,https://preneur-masr.com/news/icsb-%d9%85%d8%b...,ريادة أعمال
3,مسابقة تطوير مصر 2020 لريادة الأعمال,للعام الرابع علي التوالي تنظم شركة تطوير مصر م...,https://preneur-masr.com/news/%d9%85%d8%b3%d8%...,ريادة أعمال
4,MAGNiTT :: إستثمارت قياسية للشركات الناشئة في ...,نشرت شركة “ماجنيت MAGNiTT ” لبيانات الشركات ا...,https://preneur-masr.com/news/magnitt-%d8%a5%d...,ريادة أعمال
...,...,...,...,...
105,مؤتمر ميكروسوفت و EFE لريادة الأعمال بالأسكندرية,يقام يوم الإثنين القادم الموافق 20 أبريل 2015...,https://preneur-masr.com/news/%d9%85%d8%a4%d8%...,ريادة أعمال
106,الصندوق الإجتماعي يوقع 3 مذكرات تفاهم مع العمل...,علي هامش الإحتفال بإعتماد منظمة العمل الدولية ...,https://preneur-masr.com/news/%d8%a7%d9%84%d8%...,ريادة أعمال
107,الصناعة تدعم رواد الأعمال ب 2 مليون جنيه,قال وزير الصناعة والتجارة والمشروعات الصغيرة و...,https://preneur-masr.com/news/%d8%a7%d9%84%d8%...,ريادة أعمال
108,الرابط بين شهر رمضان المبارك و ريادة الأعمال,كل عام و أنتم بخيرها هو أجمل شهر قد أهل علينا ...,https://preneur-masr.com/articles-investigatio...,ريادة أعمال


# Science, Technology & innovation

In [27]:
data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get("https://arabic.rt.com/technology/news/")

while True:
    try:
        element = driver.find_element_by_class_name('more-button')
    except:
        print("can't find the 'more' button anymore! Great")
        break;
    else:
        driver.execute_script("arguments[0].click();", element)
        time.sleep(5)
   
#element = driver.find_element_by_css('div[class*="loadingWhiteBox"]')
#webdriver.ActionChains(driver).move_to_element(element ).click(element ).perform()
soup=BeautifulSoup(driver.page_source,'html.parser')
stories = soup.findAll("div", {"class": "element"})

l=0
for n in range(len(stories)):
    
    title = stories[n].find('h2').text
    url = 'https://arabic.rt.com/' + stories[n].find('h2').find('a').attrs['href']
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    div = soup.find("div", {"class": "text js-text js-mediator-article"})
    ps = div.findAll('p')
    content = ""
    for p in ps:
        content += p.text
    category = 'علوم وتكنولوجيا'
    if content != "" and category != "":
        add_to_json(data, title, content, url, category)
        l += 1
print(l)
save_json(data, '../Datasets/Science-Technology/RT-Online')

can't find the 'more' button anymore! Great
1000


In [28]:
#df = pd.read_csv('../Datasets/RT-Online.csv')
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/Science-Technology/RT-Online.csv", index=False)
df

,Title,Content,Link,Category
0,\n الجيش الروسي يحصل عل...,\n أعلنت مصادر في وزارة الدفاع الرو...,https://arabic.rt.com//technology/1199321-%D8%...,علوم وتكنولوجيا
1,\n اكتشاف ظاهرة هائلة و...,\n اكتشف باحثون دورة جديدة كاملة من...,https://arabic.rt.com//technology/1199366-%D8%...,علوم وتكنولوجيا
2,\n علماء يتوصلون إلى دل...,\n اكتشف باحثون للتو ما قد يكون قد ...,https://arabic.rt.com//technology/1199304-%D8%...,علوم وتكنولوجيا
3,\n ما مدى ثراء رئيس أما...,"\n تنحى مؤسس ""أمازون""، جيف بيزوس، ع...",https://arabic.rt.com//technology/1199371-%D9%...,علوم وتكنولوجيا
4,\n تقييم دراسة عن تأثير...,\n أعلن البروفيسور ألكسندر كيسلوف، ...,https://arabic.rt.com//technology/1199246-%D8%...,علوم وتكنولوجيا
...,...,...,...,...
995,\n روسيا تختبر قدرات دف...,\n نشرت وسائل إعلام روسية جانبا من ...,https://arabic.rt.com//technology/1151750-%D8%...,علوم وتكنولوجيا
996,\n أكبر جليد في إيطاليا...,\n وفقا لنتائج دراسة جديدة أجراها ع...,https://arabic.rt.com//technology/1151647-%D8%...,علوم وتكنولوجيا
997,\n ما خطورة الأسفلت الس...,\n حلل علماء من الولايات المتحدة وأ...,https://arabic.rt.com//technology/1151646-%D9%...,علوم وتكنولوجيا
998,\n محاكاة تكشف ما قد تب...,\n على الرغم من أن المادة المظلمة ت...,https://arabic.rt.com//technology/1151790-%D9%...,علوم وتكنولوجيا


In [29]:
data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get("https://asharq.com/category/%D8%AA%D9%83%D9%86%D9%88%D9%84%D9%88%D8%AC%D9%8A%D8%A7/")
        
for i in range(20):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)
    
soup=BeautifulSoup(driver.page_source,'html.parser')
media = soup.findAll("div", {"class": "gridstyled__GridCardLink-sc-1fz3w0t-20 fLSEMG"})

l=0
for n in range(len(media)):
    
    #title = media[n].find('a').text
    url = "https://asharq.com" + media[n].find('a').attrs['href']
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    if soup.find("div", {"id": "article_content_body"}) != None:
        title = soup.find("div", {"class": "articlestyled__MastheadContent-rgo6b1-1 hHQxFA"}).find("h1").text
        div = soup.find("div", {"id": "article_content_body"})
        ps = div.findAll('p')
        content = ""
        for p in ps:
            content += p.text
        category = "علوم وتكنولوجيا"
        if content != "" and category != "":
            add_to_json(data, title, content, url, category)
            l += 1
print(l)
save_json(data, '../Datasets/Science-Technology/asharq')

318


In [30]:
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/Science-Technology/asharq.csv", index=False)
df

,Title,Content,Link,Category
0,"هذا ما يستطيع ""الروبوت الكلب"" القيام به بعد تط...",أطلقت شركة بوسطن ديناميكس مجموعة من التحديثات ...,https://asharq.com/ar/3zPmSiLLvX0B1IscTwHpUu-ت...,علوم وتكنولوجيا
1,دون أسباب.. أبل تحرم مستخدمي كروم من ميزة مهمة,"بعد إطلاقها للميزة منذ أيام، سحبت ""أبل"" تحديث ...",https://asharq.com/ar/CqifnkKVQqFtESkpTFMR9-دو...,علوم وتكنولوجيا
2,خاصية جديدة لإعادة الصور والفيديوهات المحذوفة ...,أعلنت شبكة إنستغرام الاجتماعية عن ميزة جديدة ت...,https://asharq.com/ar/4JdP0xnW47B63G9rM06pf0-خ...,علوم وتكنولوجيا
3,"كوريا تسقط اتهام ""أبل"" بالاحتكار مقابل 89.7 مل...",أعلنت لجنة التجارة العادلة في كوريا الجنوبية ق...,https://asharq.com/ar/Sio6uk9wIuCGbHKeZFA4p-كو...,علوم وتكنولوجيا
4,"من بينها متجر ""آب ستور"".. تعطل جزئي في خدمات آبل",تشهد بعض خدمات شركة آبل عطلاً مفاجئاً، مثل خدم...,https://asharq.com/ar/3LHh1yuDF949fSB6yQcPYc-م...,علوم وتكنولوجيا
...,...,...,...,...
313,العمل عن بعد يرفع وتيرة الهجمات الإلكترونية,يحذر خبراء في الأمن السيبراني بحلف شمال الأطلس...,https://asharq.com/ar/5gemn4APcCaiRw3Uyl3dDr-ا...,علوم وتكنولوجيا
314,أيفون بلا شاحن.. التزام تجاه الكوكب أم حيلة تس...,"أثارت شركة ""أبل"" الأميركية ردود أفعال واسعة لد...",https://asharq.com/ar/2ckCsvkI803BOZsjMlAHiN-أ...,علوم وتكنولوجيا
315,تعطل خدمات تويتر.. والموقع: مشكلة داخلية ولا د...,اشتكى الآلاف من مستخدمي موقع التدوينات القصيرة...,https://asharq.com/ar/07sCwHQ1yhXa62C0Cj6qa-تع...,علوم وتكنولوجيا
316,"آبل تطلق ""آيفون 12"" الداعم لشبكات الجيل الخامس",أطلقت شركة آبل الأميركية، الثلاثاء، هاتفها الج...,https://asharq.com/ar/4PzY0ukU513Y54IjlEse6x-آ...,علوم وتكنولوجيا


In [45]:
data = {}
data['Article'] = []

service = Service('./chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get("https://arabic.sputniknews.com/science/")
        
  
l=0
while True:
    try:
        element = driver.find_element_by_class_name('b-btn.m-more')
    except:
        l += 1
        print("can't find the 'more' button anymore! Great")
        if l > 2:
            break;
    else:
        try:
            element.click()
            time.sleep(5)
        except:
            break;
    
soup=BeautifulSoup(driver.page_source,'html.parser')
media = soup.findAll("div", {"class": "b-stories__title"})

l=0
for n in range(len(media)):
    
    title = media[n].find('h2').find('a').text
    url = "https://arabic.sputniknews.com" + media[n].find('h2').find('a').attrs['href']
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    if soup.find("div", {"class": "b-article__text"}) != None:
        div = soup.find("div", {"class": "b-article__text"})
        ps = div.findAll('p')
        content = ""
        for p in ps:
            content += p.text
        category = "علوم وتكنولوجيا"
        if content != "" and category != "":
            add_to_json(data, title, content, url, category)
            l += 1
print(l)
save_json(data, '../Datasets/Science-Technology/sputniknews')

699


In [46]:
df = pd.json_normalize(data['Article'])
df.drop_duplicates(inplace=True)
df.to_csv("../Datasets/Science-Technology/sputniknews.csv", index=False)
df

,Title,Content,Link,Category
0,"""شرنقة"" مصرية غريبة حيرت العلماء.. المجلات الع...","وأثار اكتشاف ""مومياء طينية"" نادرة في مصر القدي...",https://arabic.sputniknews.com/society/2021020...,علوم وتكنولوجيا
1,علماء يقدمون نظرية جديدة لتفسير حدوث ظاهرة غام...,في وقت سابق، اقترح العلماء، أن تدفقات المخلفات...,https://arabic.sputniknews.com/science/2021020...,علوم وتكنولوجيا
2,"ليس ""د""... دراسة: فيتامين آخر هو المسؤول عن ""ك...",وبحسب الدراسة العلمية التركية المنشورة على بوا...,https://arabic.sputniknews.com/society/2021020...,علوم وتكنولوجيا
3,5 أعشاب سحرية يمكنها تخليصك من مشاكل الرئة وال...,"لذلك يقدم موقع ""تايمز نيوز ناو"" مجموعة من الأع...",https://arabic.sputniknews.com/science/2021020...,علوم وتكنولوجيا
4,اكتشاف مستخلص نباتي أثبت فعالية في التصدي لفير...,وتولد هذه المادة استجابة مناعية فطرية عالية ال...,https://arabic.sputniknews.com/science/2021020...,علوم وتكنولوجيا
...,...,...,...,...
4076,الكشف عن 6 أعراض السرطان لدى الرجال,قد يكون تغيير شكل الثدي أو إفرازات حلمة الثدي ...,https://arabic.sputniknews.com/science/2019021...,علوم وتكنولوجيا
4077,قمر صناعي روسي يكتشف ظواهر عجيبة في الغلاف الج...,وأوضح بانياسوك أن الغلاف الجوي للأرض يضيء برشق...,https://arabic.sputniknews.com/science/2019021...,علوم وتكنولوجيا
4078,عالم يزعم توصله إلى توقيت دخول الروح إلى الجسد...,ونشر العالم المتخصص في الطب النفسي فرضيته في ك...,https://arabic.sputniknews.com/science/2019021...,علوم وتكنولوجيا
4079,"عالم ""يدحض"" نظرية التطور وينسب النسل البشري إل...",نشر عالم الكيمياء الحيوية من الولايات المتحدة ...,https://arabic.sputniknews.com/science/2019021...,علوم وتكنولوجيا
